In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

In [8]:
# Создаём сэмпл. Стоимости перелета на рейсе в зависимости от заполнености самолета и растояния.
# Скажем так.. стоимость за перелет с них будут брать после перелета, чтобы эта модель хоть как то могла 
# напоминать настоящую ситуацию:))

n_samples = 500

distance = np.random.choice(7000, n_samples) + 1000 # Растояние в км. которое пролетает самолет за рейс
fuel = np.random.choice(3000, n_samples) + 1500 # Количество топлива которое он сжигает за перелет в галлонах
passengers = np.random.choice(250, n_samples) + 50 # Сколько людей летит на рейсе

price_per_pas = (distance  * 125) / passengers

# создаем табличку
data = pd.DataFrame({'distance': distance, 'fuel': fuel, 'passengers': passengers, 'price_per_pas': price_per_pas})
data.head(5)

,distance,fuel,passengers,price_per_pas
0,2405,3524,290,1036.637931
1,6155,3648,215,3578.488372
2,6965,3023,66,13191.287879
3,1027,2239,139,923.561151
4,1673,4285,128,1633.789062


In [9]:
from sklearn.metrics import mean_absolute_error

# Можно посмотреть будет ли зависимость от всех наших параметров и на сколько кореллируется с ценой билета)
X = data[['distance', 'fuel', 'passengers']]
y = data['price_per_pas']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['distance', 'fuel', 'passengers']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [  0.93333516  -0.08322315 -29.21047689]
Bias: 5197.306068447214
Error: 1119.3678156751455


Ну должно быть, что цена не должна зависить от заполнености самолета.

In [10]:
X = data[['distance', 'fuel']]
y = data['price_per_pas']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['distance', 'fuel']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 0.94882676 -0.08234418]
Bias: 98.33018197668571
Error: 1793.3875035145934


Ну и вот мы получили то, что надо))). Иногда мне кажется так и формируется цена на билеты)
Зависимость тут просматривается еще меньше, наверное потомучто теперь неизвесна одна переменная используемая в формуле по расчету одного билета.

In [11]:
# Создаем новый признак
data['cost_one_km'] = data['distance'] * data['price_per_pas']
data.head(5)

,distance,fuel,passengers,price_per_pas,cost_one_km
0,2405,3524,290,1036.637931,2.493114e+06
1,6155,3648,215,3578.488372,2.202560e+07
2,6965,3023,66,13191.287879,9.187732e+07
3,1027,2239,139,923.561151,9.484973e+05
4,1673,4285,128,1633.789062,2.733329e+06


In [12]:
X = data[['cost_one_km']]
y = data['price_per_pas']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['cost_one_km']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [0.00012773]
Bias: 1231.1012682197306
Error: 658.1905701929501


и вот когда мы уменьшили пространство и задали более менее понятный фактор. то ошибка стала в разы меньше.